In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
import time

training_dir = 'C:/Users/jared/Documents/GitHub/CSE450-Team/Project_4/Jared-Work/training2'

def augment_images(directory, target=50, max_images=5000):
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    for folder in os.listdir(directory):
        class_dir = os.path.join(directory, folder)
        num_images = len(os.listdir(class_dir))
        
        if num_images < max_images:
            augmented_images = 0
            images_this_second = 0
            last_second = int(time.time())  # get the current time in seconds since the epoch

            for file in os.listdir(class_dir):
                split_filename = file.split('.')[0].split('_')  # Splitting filename by underscores
                track_num = split_filename[0]  # Get track number from the filename
                img = load_img(os.path.join(class_dir, file))  
                x = img_to_array(img)  
                x = x.reshape((1,) + x.shape)  

                i = 0
                for batch in datagen.flow(x, batch_size=1, 
                                           save_to_dir=class_dir, 
                                           save_prefix=f'{track_num}_aug_{i}_', 
                                           save_format='jpg'):
                    i += 1
                    augmented_images += 1
                    images_this_second += 1

                    # Check if a second has passed
                    current_second = int(time.time())
                    if current_second > last_second:
                        print(f"Created {images_this_second} images in the last second.")
                        last_second = current_second
                        images_this_second = 0  # reset the count for the next second

                    if i >= target:
                        break  # to avoid creating more than necessary

                    # Check total number of images after every 100 augmented images
                    if augmented_images % 100 == 0:
                        num_images = len(os.listdir(class_dir))
                        if num_images >= max_images:
                            break  # stop generating new images for this folder and move on to the next one

augment_images(training_dir)